<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/total_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seeing the imperceptible: total flow

this is a notebook which is used to have a set of images move through the processing steps which can individually be found in the topic related folders.

A **batch** of images will be processed and all intermediate steps will be saved. This is not an optimized flow, yet a flow allowing the user to intervene where necessary and have updated images continue throughout the flow.

This flow is created to test with some experts and capture their feedback, it is not intended for day-to-day usage. One testing day will be organized in which this notebook will be used with some unique images.

## steps

1. Sensor dust removal
2. Image Editing (LaMa)
3. Background Removal
4. Background Recoloring: not yet
5. Clothes recoloring: not yet
6. Skin retouching
<!-- 7. Face Detection -->
7. optional: Color Corrections: not yet
8. Color Grading: 
9. Image upscaling

TODO: is it necessary to create a config file and have separate notebooks for each step? (with 1 common config generator)

In [1]:
# first we'll link a database connection:
!curl https://raw.githubusercontent.com/luca-arts/seeingtheimperceptible/main/notebooks/database_mod.py -o /content/database_mod.py --silent
from database_mod import *

link_nextcloud()

nextcloud = '/content/database/'

what's the username for nextcloud? colab
what's the password for user colab? ··········
0
Please enter the username to authenticate with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Username: Please enter the password to authenticate user colab with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Password:  


## SETUP

we'll link this instance of the machine learning flow to your name:

In [2]:
tname = 'total' #@param {type:"string"}
if(tname=='total'):
    print("Are you sure you don't want to change the name?")

Are you sure you don't want to change the name?


## Step 1: Sensor dust removal

we'll link the main input folder and write the output images in the output folder of step 1.

In [5]:
#@title imports of libraries & setting up
input_step1, output_step1 = create_io(database=nextcloud,topic=tname,library='step1_sensor_dust', input_redirect='/content/database/total/input')

#import libraries
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os, sys
!curl https://raw.githubusercontent.com/Tschucker/Python-Automatic-Sensor-Dust-Removal/main/shapedetector.py -o /content/shapedetector.py
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
from shapedetector import ShapeDetector
import imutils
from google.colab.patches import cv2_imshow

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1255  100  1255    0     0  25100      0 --:--:-- --:--:-- --:--:-- 25100


In [ ]:
#@title Set inpainting options and plot result
radius = 11 #@param {type:"slider",min:1, max:50}
flags = cv2.INPAINT_TELEA #@param ["cv2.INPAINT_TELEA","cv2.INPAINT_NS"]

def inpaint_img(img_path, img_name, output_path, radius=10, flags=cv2.INPAINT_TELEA):
  #color version
  cimg = cv2.imread(img_path)
  #grey scale image
  img = cv2.imread(img_path,0)

  #Apply Global Threshold
  m = np.mean(img, dtype=int)
  global_thresh = cv2.threshold(img,int(m/1.2),255,cv2.THRESH_BINARY_INV)[1]

  #Perform Adaptive Threshold
  adaptive_thresh_img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY_INV,19,10)

  #Image Magnification Filter Kernel
  KERNEL = np.ones((10,10), dtype=int)*10

  #Filter the thresholded images*
  img_filt = cv2.filter2D(adaptive_thresh_img,-1,KERNEL)
  #global_thresh = cv2.filter2D(global_thresh,-1,KERNEL)

  #Apply multiple times
  for i in range(2):
      KERNEL_i = np.ones((int(10),int(10)), dtype=int)*10
      img_filt = cv2.filter2D(img_filt,-1,KERNEL_i)

  #Combine Thresholds
  comb = img_filt + global_thresh

  #Find and Classify Contours of Image
  cnts = cv2.findContours(comb.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  sd = ShapeDetector()
  cimg_copy = cimg.copy()
  for c in cnts:
      # compute the center of the contour, then detect the name of the
      # shape using only the contour
      M = cv2.moments(c)
      if M["m00"] != 0:
          cX = int((M["m10"] / M["m00"]) * 1)
          cY = int((M["m01"] / M["m00"]) * 1)
          shape = sd.detect(c)
          # multiply the contour (x, y)-coordinates by the resize ratio,
          # then draw the contours and the name of the shape on the image
          if len(c) < 50:
              c = c.astype("float")
              c *= 1
              c = c.astype("int")
              cv2.drawContours(cimg_copy, [c], -1, (0, 255, 0), 2)
              cv2.putText(cimg_copy, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
  
  #Create Dust Mask
  img_mask = np.zeros((img.shape[0], img.shape[1]), dtype='uint8')
  for c in cnts:
      # compute the center of the contour, then detect the name of the
      # shape using only the contour
      M = cv2.moments(c)
      if M["m00"] != 0:
          cX = int((M["m10"] / M["m00"]) * 1)
          cY = int((M["m01"] / M["m00"]) * 1)
          shape = sd.detect(c)
          # multiply the contour (x, y)-coordinates by the resize ratio,
          # then draw the contours and the name of the shape on the image
          if len(c) < 50:
              c = c.astype("float")
              c *= 1
              c = c.astype("int")
              cv2.fillPoly(img_mask, pts=[c], color=(255,255,255))

    
  #Inpaint the image
  cimg_inpaint = cv2.inpaint(cimg, img_mask, radius, flags=flags)

  #Show and Save Final Image
  save_img_pth = os.path.join(output_path,img_name)
  cv2.imwrite(save_img_pth, cimg_inpaint)

  # plt_out = cv2.cvtColor(cimg_inpaint, cv2.COLOR_BGR2RGB)
  # return plt_out

for img_name in os.listdir(input_step1):
  print("processing ",img_name)
  img_path = os.path.join(input_step1,img_name)
  inpaint_img(img_path, img_name, output_step1, radius=radius, flags=flags)

43.jpg /content/database/total/input
44.jpg /content/database/total/input
45.jpg /content/database/total/input
46.jpg /content/database/total/input
47.jpg /content/database/total/input
48.jpg /content/database/total/input
49.jpg /content/database/total/input
5.jpg /content/database/total/input
50.jpg /content/database/total/input


### verification

Now it's time to go to the database and verify the results. If needed we adapt the images locally. 

TODO: how easily can we **sync** the images to allow the experts to intervene? Is it possible with nextcloud?

## step 2: Minor retouching: image editing LaMa 
Once verified we want to continue with the next step, being image editing (LaMa model)

therefore we link the output folder of previous step to the inputfolder of this step.

In [ ]:
input_step2, output_step2 = create_io(database=nextcloud,topic=tname,library='step2_lama', input_redirect=output_step1)

In [ ]:
#@title imports of libraries & setting up
root_path2 = '/content/lama'

# clone the repository
if not os.path.exists(root_path2):
  !git clone https://github.com/saic-mdal/lama {root_path2}
# Set up the environment
print('\n> Install dependencies')
!pip install -r lama/requirements.txt --quiet
# do we need the wget?
!pip install wget --quiet

# download the model
print('\n> Download the model')
!curl -L $(yadisk-direct https://disk.yandex.ru/d/ouP6l8VJ0HpMZg) -o {root_path2}/big-lama.zip
# todo check where the model is unzipped
!unzip {root_path2}/big-lama.zip

# fixing openCV
print('>fixing opencv')
!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet

/content/database/total/step1_sensor_dust /content/database/total/step2_<...>


In [ ]:
import base64, os
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
import wget
from shutil import copyfile
import shutil

#@ title: imports & helper functions

canvas_html = """
<style>
.button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}
</style>
<canvas1 width=%d height=%d>
</canvas1>
<canvas width=%d height=%d>
</canvas>

<button class="button">Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')

var canvas1 = document.querySelector('canvas1')
var ctx1 = canvas.getContext('2d')


ctx.strokeStyle = 'red';

var img = new Image();
img.src = "data:image/%s;charset=utf-8;base64,%s";
console.log(img)
img.onload = function() {
  ctx1.drawImage(img, 0, 0);
};
img.crossOrigin = 'Anonymous';

ctx.clearRect(0, 0, canvas.width, canvas.height);

ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}

canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}

var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

def draw(imgm, filename='drawing.png', w=400, h=200, line_width=1):
  display(HTML(canvas_html % (w, h, w,h, filename.split('.')[-1], imgm, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)


In [ ]:
for i in os.listdir(input_step2):
  fname = os.path.join(input_step2,i)

  image64 = base64.b64encode(open(fname, 'rb').read())
  image64 = image64.decode('utf-8')

  print(f'Will use {fname} for inpainting')
  img = np.array(plt.imread(f'{fname}')[:,:,:3])

  draw(image64, filename=f"./{fname.split('.')[1]}_mask.png", w=img.shape[1], h=img.shape[0], line_width=0.04*img.shape[1])
  #@title Show a masked image and save a mask

  # plt.rcParams["figure.figsize"] = (15,5)
  # plt.rcParams['figure.dpi'] = 200
  # plt.subplot(131)
  # with_mask = np.array(plt.imread(f"./{fname.split('.')[1]}_mask.png")[:,:,:3])
  # mask = (with_mask[:,:,0]==1)*(with_mask[:,:,1]==0)*(with_mask[:,:,2]==0)
  # plt.imshow(mask, cmap='gray')
  # plt.axis('off')
  # plt.title('mask')
  # plt.imsave(f"./{fname.split('.')[1]}_mask.png",mask, cmap='gray')

  # plt.subplot(132)
  # img = np.array(plt.imread(f'{fname}')[:,:,:3])
  # plt.imshow(img)
  # plt.axis('off')
  # plt.title('img')

  # plt.subplot(133)
  # img = np.array((1-mask.reshape(mask.shape[0], mask.shape[1], -1))*plt.imread(fname)[:,:,:3])
  # _=plt.imshow(img)
  # _=plt.axis('off')
  # _=plt.title('img * mask')
  # plt.show()

  # os.makedirs('/content/output')

  print('Run inpainting')
  if '.jpeg' in fname:
    !PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py model.path=$(pwd)/big-lama indir=$(pwd)/data_for_prediction outdir={output_step2} dataset.img_suffix=.jpeg > /dev/null
  elif '.jpg' in fname:
    !PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py model.path=$(pwd)/big-lama indir=$(pwd)/data_for_prediction outdir={output_step2}  dataset.img_suffix=.jpg   > /dev/null
  elif '.png' in fname:
    !PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py model.path=$(pwd)/big-lama indir=$(pwd)/data_for_prediction outdir={output_step2}  dataset.img_suffix=.png > /dev/null
  else:
    print(f'Error: unknown suffix .{fname.split(".")[-1]} use [.png, .jpeg, .jpg]')

  plt.rcParams['figure.dpi'] = 200

  print(os.listdir(output_step2))

# plt.imshow(plt.imread(f"/content/output/{fname.split('.')[1].split('/')[2]}_mask.png"))
# _=plt.axis('off')
# _=plt.title('inpainting result')
# plt.show()

# fname = None

## Step 3: background removal

Again, verify the outcome of step 2. 

Now we'll subtract the background using the ModNet model.

<!-- Or use the other model if better -->



In [ ]:
input_step3, output_step3 = create_io(database=nextcloud,topic=tname,library='step3_bg_removal', input_redirect=output_step2)

## Step 4: Background recoloring

In [ ]:
input_step4, output_step4 = create_io(database=nextcloud,topic=tname,library='step4_bg_coloring', input_redirect=output_step3)

## Step 5: Clothes recoloring

In [ ]:
input_step5, output_step5 = create_io(database=nextcloud,topic=tname,library='step5_clothes_coloring', input_redirect=output_step4)

In [ ]:
import shutil
# as we're not implementing clothes recoloring yet, we copy the folders to surpass this step
shutil.copytree(output_step4,output_step5, dirs_exist_ok=True)

## Step 6: Skin retouching

we'll implement the retouchML library

In [ ]:
input_step6, output_step6 = create_io(database=nextcloud,topic=tname,library='step6_skin_retouch', input_redirect=output_step5)

## Step 7: Color Corrections



In [ ]:
input_step7, output_step7 = create_io(database=nextcloud,topic=tname,library='step6_color_corrections', input_redirect=output_step6)

## Step 8: Color Grading

we're implementing the deep preset library.

In [ ]:
input_step8, output_step8 = create_io(database=nextcloud,topic=tname,library='step7_color_grading', input_redirect=output_step7)

## Step 9: image upscaling

This is to be implemented **if** necessary?

In [ ]:
input_step9, output_step9 = create_io(database=nextcloud,topic=tname,library='step8_image_upscaling', input_redirect=output_step8)